# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

### Ознакомимся с содержимым:

In [2]:
df0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
df0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
df1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
df1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
df2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')
df2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


### Проверим информацию о данных:

In [5]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
print('Количество дубликатов в НУЛЕВОМ регионе:', df0.duplicated().sum())
print('Количество дубликатов в ПЕРВОМ регионе:', df1.duplicated().sum())
print('Количество дубликатов во ВТОРОМ регионе:', df2.duplicated().sum())

Количество дубликатов в НУЛЕВОМ регионе: 0
Количество дубликатов в ПЕРВОМ регионе: 0
Количество дубликатов во ВТОРОМ регионе: 0


**Первичные выводы: дубликаты и пропуски отсутствуют, тип значений в столбцах - float64. Столбец id с типом object нам не понадобится в дальнейших целях.**

## Обучение и проверка модели

### Разделение данных на обучающую и валидационную выборку:

In [9]:
features0 = df0.drop(columns=['product', 'id']) 
target0 = df0['product'] 
features1 = df1.drop(columns=['product', 'id']) 
target1 = df1['product']
features2 = df2.drop(columns=['product', 'id']) 
target2 = df2['product']

In [10]:
features_train0, features_valid0, target_train0, target_valid0 = train_test_split(
    features0, target0, test_size=0.25, random_state=12345)
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    features1, target1, test_size=0.25, random_state=12345)
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    features2, target2, test_size=0.25, random_state=12345)

print(features_train0.shape) #проверка размера 
print(features_valid0.shape) #проверка размера 

(75000, 3)
(25000, 3)


### Обучение модели линейной регрессии:

In [11]:
model0 = LinearRegression() 
model0.fit(features_train0, target_train0) 
predictions_valid0 = model0.predict(features_valid0)
result0 = (mean_squared_error(target_valid0, predictions_valid0))**0.5 
print('НУЛЕВОЙ регион')
print('Cредний запас предсказанного сырья:', predictions_valid0.mean())
print('RMSE модели:', result0)

НУЛЕВОЙ регион
Cредний запас предсказанного сырья: 92.59256778438035
RMSE модели: 37.5794217150813


In [12]:
model1 = LinearRegression()
model1.fit(features_train1, target_train1) 
predictions_valid1 = model1.predict(features_valid1) 
result1 = (mean_squared_error(target_valid1, predictions_valid1))**0.5
print('ПЕРВЫЙ регион')
print('Cредний запас предсказанного сырья:', predictions_valid1.mean())
print('RMSE модели:', result1)

ПЕРВЫЙ регион
Cредний запас предсказанного сырья: 68.728546895446
RMSE модели: 0.893099286775617


In [13]:
model2 = LinearRegression()
model2.fit(features_train2, target_train2) 
predictions_valid2 = model2.predict(features_valid2) 
result2 = (mean_squared_error(target_valid2, predictions_valid2))**0.5
print('ВТОРОЙ регион')
print('Cредний запас предсказанного сырья:', predictions_valid2.mean())
print('RMSE модели:', result2)

ВТОРОЙ регион
Cредний запас предсказанного сырья: 94.96504596800489
RMSE модели: 40.02970873393434


**Выводы: <br/> 
Минимальная ошибка в ПЕРВОМ регионе, но в нем самый низкий средний запас предсказанного сырья.  <br/>
Максимальный запас предсказанного сырья во ВТОРОМ регионе**

## Подготовка к расчёту прибыли

In [14]:
BUDGET = 10**10
INCOME_PER_BARREL = 450000
TOTAL_WELL_QTY = 500
SELECTED_WELL_QTY = 200
LOSS_POSSIBILITY = 0.025
BREAKEVEN_POINT = (BUDGET/INCOME_PER_BARREL/SELECTED_WELL_QTY)
print('Объем добычи скважины для достижения безубыточности:', BREAKEVEN_POINT)

Объем добычи скважины для достижения безубыточности: 111.11111111111111


**Вспоминаем средний запас предсказанного сырья:**

In [15]:
print(predictions_valid0.mean(), predictions_valid1.mean(), predictions_valid2.mean())

92.59256778438035 68.728546895446 94.96504596800489


Рассмотрим данные детально:

In [16]:
df0['product'].describe()

count    100000.000000
mean         92.500000
std          44.288691
min           0.000000
25%          56.497507
50%          91.849972
75%         128.564089
max         185.364347
Name: product, dtype: float64

In [17]:
df1['product'].describe()

count    100000.000000
mean         68.825000
std          45.944423
min           0.000000
25%          26.953261
50%          57.085625
75%         107.813044
max         137.945408
Name: product, dtype: float64

In [18]:
df2['product'].describe()

count    100000.000000
mean         95.000000
std          44.749921
min           0.000000
25%          59.450441
50%          94.925613
75%         130.595027
max         190.029838
Name: product, dtype: float64

In [19]:
print("25% скважин в нулевом регионе содержат больше сырья, чем ",
      round(df0["product"].quantile(0.75),2))
print("16% скважин в нулевом регионе содержат больше сырья, чем ",
      round(df1["product"].quantile(0.84),2))
print("25% скважин в нулевом регионе содержат больше сырья, чем ",
      round(df2["product"].quantile(0.75),2))

25% скважин в нулевом регионе содержат больше сырья, чем  128.56
16% скважин в нулевом регионе содержат больше сырья, чем  134.77
25% скважин в нулевом регионе содержат больше сырья, чем  130.6


**Несмотря на то, что среднее значение по регионом не дотягивало до необходимого минимального количества баррелей нефти для безубыточной разработки, 25% (2 500) скважин нулевого и второго региона и 16% (1 600) скважин первого региона содержат необходимое количество сырья.**

## Расчёт прибыли и рисков 

In [23]:
def profit(true_target, pred_target):
    sort_Series = pd.Series(pred_target).sort_values(ascending=False)[:SELECTED_WELL_QTY]
    true_target_sort = (true_target
                         .reset_index(drop = True)[sort_Series.index])
    sum_true = true_target_sort.sum() 
    return round((sum_true * INCOME_PER_BARREL) - BUDGET,2)
state = np.random.RandomState(12345)

In [24]:
def confidence_interval(true_target,pred_target):
    samples = []
    for i in (range(1000)):
        sample = pd.Series(pred_target).sample(n = TOTAL_WELL_QTY, replace=True, random_state=state)
        samples.append(profit(true_target,sample))
    samples = pd.Series(samples)
    print("Средняя прибыль:",samples.mean())
    print(samples.apply(lambda x: x < 0).sum()/len(samples)*100,"%")
    
    lower = samples.quantile(0.025)
    upper = samples.quantile(0.975)
    return round(lower,2), round(upper,2)

In [25]:
print("Для НУЛЕВОГО региона:")
print("95% доверительный интервал:",
      confidence_interval(target_valid0,pd.Series(predictions_valid0)))
print()
print("Для ПЕРВОГО региона:")
print("95% доверительный интервал:",
      confidence_interval(target_valid1,pd.Series(predictions_valid1)))
print()
print("Для ВТОРОГО региона:")
print("95% доверительный интервал:",
      confidence_interval(target_valid2,pd.Series(predictions_valid2)))

Для НУЛЕВОГО региона:
Средняя прибыль: 396164984.80228
6.9 %
95% доверительный интервал: (-111215545.89, 909766941.55)

Для ПЕРВОГО региона:
Средняя прибыль: 461155817.2772
0.7000000000000001 %
95% доверительный интервал: (78050810.75, 862952060.26)

Для ВТОРОГО региона:
Средняя прибыль: 392950475.17066
6.5 %
95% доверительный интервал: (-112227625.38, 934562914.55)


**Вывод: <br/>
По результатам работы для разработки месторождения имеет смысл предложить Первый регион, т.к.:  <br/>**
* **по условию максимального допустимого уровня рисков -2,5% проходит Первый регион.** <br/>
* **в данном регионе максимальная средняя прибыль.**